In [1]:
import pandas as pd
import numpy as np
import os
import string
import sys
import re
import datetime as dt
import import_ipynb
#변수명 추출
from varname import nameof
#os정렬
import natsort
from tqdm import tqdm
# from standard_func import *
# import standard_func_py as sf
# now = dt.datetime.now().strftime('%y%m%d_%H%M%S')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# 진행 여부 함수
def yes_or_no(question):
    while "the answer is invalid":
        reply = str(input(question+' (y/n): ')).lower().strip()
        if reply[0] == 'y':
            return True
        if reply[0] == 'n':
            return False
        else:
            print("실패: 'y' 혹은 'n' 를 입력해 주세요")

### 함수

In [28]:
# blnk/검색 로직

def blnk_chk(data):
    cnt_chk = len(data)

    chk_nll_list = []
    chk_nll_cnt_list = []
    # null값 체크(개수 파악)
    for i,j in zip(range(len(data.columns)), data.columns):
        indv_cnt = data.loc[data[j] != '', j]
        if len(data[j]) != len(indv_cnt):
            print(j + ": " + str(len(indv_cnt)))
            chk_nll_list.append(j)
            chk_nll_cnt_list.append(len(indv_cnt) - cnt_chk)
    print("================================")
    if len(chk_nll_list) == 0:
        print("모든 값이 입력되었습니다.")
    elif len(chk_nll_list) > 1 :
        print("blnk(null)이 포함된 컬럼이 탐지되었습니다.")
        print(chk_nll_list)
        print("================================")

    globals()["cnt_chk_out"] = pd.DataFrame(zip(chk_nll_list, chk_nll_cnt_list), columns = ["NULL포함컬럼명", "NULL개수"])
    print(cnt_chk_out)
    
    print("================================")
    print("null 인덱스를 임시 보관합니다.")
    print("================================")
    nll_chk_row = cnt_chk_out.iloc[:,0].to_list()
    for i, j in zip(range(len(nll_chk_row)), nll_chk_row):
        globals()["null_{}".format(i)] = data.loc[data[j] == '', j].index
        print("널인덱스보관_" + str(j) + "(변수명) -> null_{}".format(i))
        
# 스페이스 제거 함수
def rmv_spce(data, opt, *col):
    if opt == 'all': 
        for i in data.columns:
            data[i] = data[i].str.replace(" ", "")
            print(str(i) + ":\t 스페이스 제거 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].str.replace(" ", "")
        print(col + ":\t 스페이스 제거 성공")
#         print(type(col))

# 불필요 컬럼 빼기 - 이름으로 빼기
def brief_col(data, excpt_col_list, name):
    col_list = data.columns
    selected_col_list = [x for x in col_list if x not in excpt_col_list]
    print(selected_col_list)
    globals()["{}".format(name)] = data[selected_col_list]
    
# 불필요 컬럼 빼기 - 순서로 빼기 
# tmp_col_list = [5,6,7,8]
# tmp_tbl_list = [2,3,4,5]
def brief_col_num(data, excpt_col_list, name):
    col_list = data.columns
    col_list_num = [*range(len(col_list))]
    selected_col_list_num = [x for x in col_list_num if x not in excpt_col_list]
    selected_col = col_list[selected_col_list_num]
    print(selected_col)
    globals()["{}".format(name)] = data[selected_col]

# 컬럼명(순서) 선택 및 변경
def cor_col(data, select_col, modify_col, name):
    # 컬럼명(순서) 선택
    globals()["{}".format(name)] = data[select_col]
    # 컬렴명 변경
    globals()["{}".format(name)].columns = modify_col
    
# null 공백처리 함수
def nll_to_blnk(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 Null 공백 처리]")
        for i in data.columns:
            data[i] = data[i].fillna("")
            print(str(i) + ":\t Null 공백 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data[col] = data[col].fillna("")
        print(col + ":\t  공백 처리 성공")

# 컬럼 고유값 수 비교
def cmpar(data, col1, col2):
    crtr_1 = len(data[col1].unique()) - 1
    crtr_2 = len(data[col2].unique()) - 1
    if crtr_1 == crtr_2:
        print('[정상] 용어명과 분리된용어명의 고유값 수가 같습니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
    else:
        print('[오류] 용어명과 분리된용어명의 고유값 수가 다릅니다.= ' + str(crtr_1) + " vs " + str(crtr_2))
        
# 공백 null처리 함수
def blank_to_nll(data, opt, *col):
    if opt == 'all':
        print("[모든 컬럼 공백 Null 처리]")
        for i in data.columns:
            data.loc[data[i] == '', i] = None
            print(str(i) + ":\t Null 처리 성공")
    elif opt == 'col':
        # 함수 가변인자는 스트링(가나다라)이 아니라 튜플(가.나.다.라)이므로 스트링으로 join 시켜줘야 한다!!!@
        col = ''.join(col)
        data.loc[data[col] == '', col] = None
        print(col + ":\t  Null 처리 성공")
        
def row_nll_rmv(data, name):
    ### 값이 한개라도 있을 경우만 카운트 ###

    data_cnt = len(data)
    notna_list = []
    for i in range(data_cnt):    
        # row 기준 널 체크(값이 한개라도 있을 경우 포함)
        if data.iloc[i].notna().sum() > 1:
            notna_list.append(i)
    globals()["{}".format(name)] = globals()["{}".format(data)].loc[notna_list]
    print(notna_list)
    print(data.shape)
    
# blank row 삭제
def blnk_row_rmv(data, opt, *col):
    if opt == 'all':
        data_cnt = len(data)
        notna_list = []
        for i in range(data_cnt):
            if (data.iloc[i,:] != '').sum() > 1:
                notna_list.append(i)
        globals()["blnk_rmv_all".format(col)] = data.iloc[notna_list]
        print(globals()["blnk_rmv_all".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")
    elif opt == 'col':
        data_cnt = len(data)
        notna_list = []
        col = ''.join(col)
        for i in range(data_cnt):
            if (data.loc[i,col] != ''):
                notna_list.append(i)
        globals()["blnk_rmv_{}".format(col)] = data.iloc[notna_list].copy()
        print(globals()["blnk_rmv_{}".format(col)].shape)
        print("OUT: blnk_rmv_all, blnk_rmv_{}")
    
# 파이썬 로드후 파일로드
def read_grip_file(data, path, file_name):
    os.system(path + file_name + ".xlsx")
    globals()[file_name] = pd.read_excel(path + file_name + ".xlsx", index_col = 0, na_filter=False)

# 파이썬 로드 및 파일 저장 및 파일 실행
def save_grip_file(data, path, file_name):
    globals()[file_name] = data
    data.to_excel(path + file_name + ".xlsx")
    os.system(path + file_name + ".xlsx")

### 추가 1) 메타데이터 띄움처리 전 중복 제거..
### 추가 2) 한글명 notnull vs null 분리하여 notnull만 관리(null: 파일 생성 및 기입 요청)
### 추가 3.1) 한글 용어 점검 기능: 한글명+영문명+숫자조합 가능), 그외 모두 제거
### 추가 3.2) 한글 단어 점검 기능: 한글명(숫자조합 가능), 영문명만(숫자조합 가능) 그외 모두 제거
### 추가 4) 테이블기준 상세항목 정의하기.. 

In [3]:
# 표준용어 구분자 변환 점검
print(len('가 나 다'.split(" ")))
'가 나 다'.split(" ")

3


['가', '나', '다']

In [4]:
'가 나 다'.count(" ")

2

# 한글명은 notNull이어야 함

작업내용:
1. 중복제거(unique): 작업 효율성 제고
2. 개수 매칭 -> (모든 값 채우기)
3. 순서 매칭: ID 부여, 한글/영문
4. 재조립( 한글명 기준)

In [5]:

테이블 한글명 + 컬럼 한글명 한줄 생성
도메인 한글명 - > 도메인명 생성

SyntaxError: invalid syntax (860564645.py, line 1)

# 용어분리

In [6]:
# 파일 불러오기
data_name = "input_test_KORAIL"
sheet_name = "테이블컬럼 입력항목 통계조사"
input_path = os.path.join(os.getcwd(), "input\\")
output_path = os.path.join(os.getcwd(), "output\\")
input_meta = pd.read_excel(input_path + data_name +".xlsx", sheet_name=sheet_name, header=0, na_filter=False)

In [7]:
# 테이블 따로(명명규칙)
input_meta['분리된테이블한글명'] = input_meta['테이블한글명*']

# 컬럼 따로(명명규칙)
input_meta['분리된컬럼한글명'] = input_meta['컬럼한글명*']

# 컬럼명부터

In [36]:
# 원본
input_meta_sepa_col = pd.DataFrame(input_meta['분리된컬럼한글명'])
print(input_meta_sepa_col.shape)
# 고유값-스페이스 대상
input_meta_sepa_col_unq = input_meta_sepa_col['분리된컬럼한글명'].unique()

(18741, 1)


In [42]:
input_meta_sepa_col_unq

array(['테이블명', '다음시퀀스번호', '설정ID', ..., '거래단말기정보', '패널티금액', '패널티코드'],
      dtype=object)

In [ ]:
# 고유값-스페이스 대상
input_meta_sepa_col_unq = input_meta_sepa_col['분리된컬럼한글명'].unique()
input_meta_sepa_col_unq = pd.DataFrame(zip(input_meta_sepa_col_unq,input_meta_sepa_col_unq), columns = ['원본', '구분자입력'])
print(input_meta_sepa_col_unq.shape)
input_meta_sepa_col_unq

# 엑셀 아웃 및 실행

In [32]:
# 파이썬 로드 및 파일 저장 및 파일 실행
def save_grip_file(data, path, file_name):
    globals()[file_name] = data
    data.to_excel(path + file_name + ".xlsx")
    os.system(path + file_name + ".xlsx")

In [ ]:
save_grip_file(input_meta_sepa_col_unq, output_path, "input_meta_sepa_col_unq")

In [65]:
print(dir)

<built-in function dir>


In [63]:
def test(data, nameof(data2):
    data2 = nameof(data2)
    print(nameof(data))
    print(data2)
test(input_meta_sepa_col_unq, input_meta_sepa_col_unq)

SyntaxError: invalid syntax (2787874715.py, line 1)

In [45]:
# 파이썬 로드 및 파일 저장 및 파일 실행
def save_grip_file(data, path):
    file_name = nameof(data)
    globals()[file_name] = data
    data.to_excel(path + file_name + ".xlsx")
    os.system(path + file_name + ".xlsx")

In [46]:
save_grip_file(input_meta_sepa_col_unq, output_path)

AttributeError: 'numpy.ndarray' object has no attribute 'to_excel'

In [ ]:
# 로드 및 파일실행
read_grip_file(dict_nan_unq, output_path, "dict_nan_unq")
dict_nan_unq

In [16]:
b = input_meta_sepa_col.merge(a, left_on='분리된컬럼한글명', right_on='원본', how='left')
b

,분리된컬럼한글명,원본,구분자입력
0,테이블명,테이블명,테이블명
1,다음시퀀스번호,다음시퀀스번호,다음시퀀스번호
2,설정ID,설정ID,설정ID
3,서비스ID,서비스ID,서비스ID
4,설정키,설정키,설정키
...,...,...,...
18736,고객ID,고객ID,고객ID
18737,등록자ID,등록자ID,등록자ID
18738,등록일시,등록일시,등록일시
18739,변경자ID,변경자ID,변경자ID


'승하차구분코드'

In [14]:
pd.DataFrame([input_meta_sepa_col_unq, input_meta_sepa_col_unq])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,3006,3007,3008,3009,3010,3011,3012,3013,3014,3015,3016,3017,3018,3019,3020,3021,3022,3023,3024,3025,3026,3027,3028,3029,3030,3031,3032,3033,3034,3035,3036,3037,3038,3039,3040,3041,3042,3043,3044,3045,3046,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056,3057,3058,3059,3060,3061,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071,3072,3073,3074,3075,3076,3077,3078,3079,3080,3081,3082,3083,3084,3085,3086,3087,3088,3089,3090,3091,3092,3093,3094,3095,3096,3097,3098,3099,3100,3101,3102,3103,3104,3105,3106,3107,3108,3109,3110,3111,3112,3113,3114,3115,3116,3117,3118,3119,3120,3121,3122,3123,3124,3125,3126,3127,3128,3129,3130,3131,3132,3133,3134,3135,3136,3137,3138,3139,3140,3141,3142,3143,3144,3145,3146,3147,3148,3149,3150,3151,3152,3153,3154,3155,3156,3157,3158,3159,3160,3161,3162,3163,3164,3165,3166,3167,3168,3169,3170,3171,3172,3173,3174,3175,3176,3177,3178,3179,3180,3181,3182,3183,3184,3185,3186,3187,3188,3189,3190,3191,3192,3193,3194,3195,3196,3197,3198,3199,3200,3201,3202,3203,3204,3205,3206,3207,3208,3209,3210,3211,3212,3213,3214,3215,3216,3217,3218,3219,3220,3221,3222,3223,3224,3225,3226,3227,3228,3229,3230,3231,3232,3233,3234,3235,3236,3237,3238,3239,3240,3241,3242,3243,3244,3245,3246,3247,3248,3249,3250,3251,3252,3253,3254,3255
0,테이블명,다음시퀀스번호,설정ID,서비스ID,설정키,기준구분코드,변경일시,설정값,삭제여부,관리자ID,SYSTEM_ID,종료일시,시작일시,CONF_VAL1,CONF_VAL2,등록자ID,등록일시,변경자ID,,사용여부,설명,파일ID,파일구분코드,파일명,파일위치,시스템ID,파일구분,교통수단코드,내용,구분(R:철도 / B:버스 / T:택시),공통코드ID,공통코드영문명,공통코드한글명,상위공통코드ID,사용여부,공통코드정의설명,공통코드값,운영종료일자,운영시작일자,공통코드값명,공통코드값약어명,공통코드값영문명,공통코드값영문약어명,화면정렬순서,인터페이스공통코드ID,인터페이스공통코드값,적용시작일자,적용종료일자,연계ID,기관코드,일련번호,교통수단명,교통사업자ID,교통사업자명,가맹점ID,가맹점명,가맹점분류코드,가맹점분류명,구간분류코드,송신 연계ID,카드사코드,COFIG ID,내부관리 파일포맷,원 수신 파일명,원 수신 파일포맷,원 수신 일자,원 송신 파일명,원 송신 파일포맷,원 송신 일자,원 파일 여부,NOTE,기관명,엽업소코드,엽업소명,보고서분류코드_SAP,운영상태여부,보고서분류명,교통수단코드명,지역구분코드,지역구분명,XROIS역코드,역명,지역본부코드,지역본부명,청구일자,수신파일명,파일레코드일련번호,거래고유번호,거래일자,거래시각,교통카드구분코드,카드번호암호화,ALIAS번호,카드사용자유형코드,KS_NT_EP,알고리즘ID,거래유형코드,KS_VK_IND_EP,전자화폐사업자코드,거래이전카드잔액,거래이후카드잔액,거래금액,SAM번호,KS_NT_PSAM,KS_NC_PSAM,KS_NI_PSAM,KS_TOT_SAM,KS_SIGN_IND,승하차구분코드,출발정류장코드,객차출발일시,도착정류장코드,객차도착일시,거래단말기ID,환승구분코드,환승누적횟수,환승소요시간,노선ID,노선번호,철도회원번호,반환금액,승차권발매일련번호,일반인원수,일반거래금액,대학생인원수,대학생거래금액,학생인원수,학생거래금액,소인인원수,소인거래금액,기타인원수,기타거래금액,여정번호,반환번호,원권번호,KS_NT_PSAM2,정산일자,카드회사코드,정산상태코드,서명인증상태코드,반송일자,반송사유코드,재처리일자,재처리사유코드,매입정산일자,매입정산상태코드,기관송신일시,기관송신파일명,송신파일레코드일련번호,"N(초기값),C(연동시)",등록시간,가맹점수신파일명,거래지점번호,충전금액,KS_ALG_EP,KS_VK_EP,KS_NT_SAM,거래이전SAM잔액,거래이후SAM잔액,LSAM전자화폐사업자코드,KS_LSIGN_IND,거래수수료율내용,충전수수료,결제유형코드,충전상태구분코드,충전거래구분코드,매입청구일자,영업일자,전송일자,고객관리번호,회원카드번호,선불카드번호,포인트,등록자 ID,송수신구분코드,CRUD 코드,진행상태코드,처리상태 상세,적재일시,처리일시,데이타 순번,사원번호,발령일자,종료일자,퇴직구분,성명,소속,소속명,부서코드,부서명,재직구분,재직상태명,직책,직책명,직급,직급명,사업영역,변경일자,등록일자,가맹점ID명,페널티코드,페널티금액,집계유형코드,집계유형코드명,상세집계코드,상세집계코드명,승하차구분코드명,레일플러스카드종류코드,후불카드회사코드,고객ID,회원유형코드,카드발급일자,카드발급시각,레일플러스카드종류코드명,구분코드,청구건수,청구금액,총청구건수,총청구금액,총반환청구건수,총반환청구금액,총반송건수,총반송금액,총재처리건수,총재처리금액,총정산건수,총정산금액,일련번호(SQ_RP_ATS),영업일,정산구분코드,정산건수,금액,수수료,부가세,은행코드,계좌번호,계좌소유자명,사업자 주소1,사업자 주소2,사업자 전화번호,사업자 우편번호,사업자 전자메일 주소,사업자 번호,대표자 이름,업종,업태,"EAI송신여부(Y,N)",성공여부,처리상세코드,처리상세메세지,...,(환불)가맹점 수수료,펌뱅킹 처리 대표기관코드,(결제사용)펌뱅킹 이용기관코드,(결제사용)펌뱅킹 하위 이용기관코드,(결제사용)펌뱅킹 

In [15]:
pd.DataFrame(zip(input_meta_sepa_tbl_uniq, input_meta_sepa_tbl_uniq), columns = ['원본', '구분자입력'])

NameError: name 'input_meta_sepa_tbl_uniq' is not defined

In [40]:
pd.DataFrame([input_meta_sepa_tbl_uniq, input_meta_sepa_tbl_uniq])

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805
0,테이블별시퀀스정보,서비스설정이력,서비스설정정보,서비스설정정보이력,서비스설정리스트,서비스설정리스트이력,SVC_LIST,SVC_LIST_HIS,SVC_SKIP_LIST,대중교통정보,공통코드목록,공통코드,인터페이스공통코드매핑,가맹점ID맵핑정보,영업소 관리,레일플러스운영현황기준정보,XROIS역코드(본부),0,SR지불거래_수신,SR지불결과_송신,SR_충전거래_수신,SR_충전거래_송신,코레일마일리지전환내역,EAI_KOVIS사원정보수신,레일플러스거래내역전송,레일플러스발급내역전송,SR지불정보_수신,SR지불정보_송신,EAI_KOVIS_레일플러스정산내역(가맹점),EAI_KOVIS_레일플러스정산내역(개인환불),EAI_KOVIS_정산처리결과수신,간선열차정보연계,KOVIS-KOTRIS 제로페이 정보수신,제로페이 세금계산서 이미지,제로페이 세금계산서 상세,카드원장이력,LSAM잔액변경이력,SAM원장이력,고객이력,파기이력관리,법인회원추가정보이력,법정대리인이력,가입고객이력,1:1 문의 정보,이중화 배치 적용 정보,신한판배치로그,자동충전 이력,코레일마일리지전환이력,자동선물설정정보(로그),R+ 카드정보 변경이력,해지 사용자 개인정보,공지사항 정보,온라인 지불결제,자동충전 이력정보,푸시 발송 정보,푸시 등록 정보,온라인 환불정보,사용자 환불 요청,사용자 환불 인증(FCS),카드정보변경이력,충전정보,파라미터 갱신 정보\t,재발급 잔액정보,단말 이력 정보,모바일 결제 정보,R+ hipass 결제 정보,발급통계,환불통계,충전통계,충전통계상세,이용내역통계,이용내역통계상세,일일실적통계,BL_PL송신이력,BL_PL수신이력,레일플러스BL전송이력,도로공사_BL카드번호,승인거래이력_가맹점,승인거래이력_VAN,잡스케줄작업이력,운영자이력,시스템메뉴이력,가맹점이력,가맹점카테고리이력,기준항목코드이력,업무별IP관리이력,시스템정보이력,파일송수신이력,전문파일업무코드이력,파일건수금액정보이력,예치금수동입금이력,프로그램실행이력,프로그램실행오류이력,레일플러스 카드별 이상거래 이력(잔액이상분만),제로페이 배치파일 송수신 내역,동해선지불충전환불거래수집파일오류,동해선승하차페어링오류,전문송수신로그,마일리지이력정보,API호출로그,앱카드 전문호출 로그,온라인충전로그,직전충전취소로그,온라인환불로그,동해선일별카드사별집계,동해선반송내역집계,공항철도지급수취역별차액정산내역집계_1회권,신분당선지급수취역별차액정산내역집계_1회권,용인경전철지급수취역별차액정산내역집계_1회권,의정부경전철지급수취역별차액정산내역집계_1회권,후불카드사별매입반송내역집계,선불카드사별매입반송내역집계,도시철도지불거래승하차집계,도시철도지불거래자연락구간수집집계,도시철도지불거래정산기수집집계,도시철도지불거래승객유형별패널티집계,시간대별지불거래집계,승객유형별지불거래집계,지불거래교통수단별환승집계,가맹점유형별지불거래집계,일별통행통계,일별유입통계,일별유출통계,광역외일별통행통계,광역외일별유입통계,광역외일별유출통계,온라인충전집계,자동충전집계,충전유형별집계,시간대별충전집계,환불유형별집계,배분금액검증집계,판매유형별집계,레일플러스카드별일단위거래집계,고객소득공제용월별집계,레일플러스선수금집계,레일플러스카드고객별일단위지불거래집계,시간대별단말기지불거래통계,정기권_1회권 시간대별승하차페어 통계,LSAM환불집계,LSAM정기권충전집계,LSAM한도충전집계,LSAM충전유형별집계,전일거래기준충전현황,SAM재고집계,카드재고집계,가맹점별카드재고집계,카드사별지불매입청구집계,가맹점별지불거래정산집계,가맹점별반환거래정산집계,철도지불거래정산집계,지불거래일별수수료집계,오프라인충전정산집계,한도충전정산집계,타카드충전거래매입집계,충전차액정산집계,코레일일별실적상세내역_임시,LSAM교체정산집계,오프라인환불정산집계,카드고객판매정산집계,가맹점일별수수료집계,가맹점월별수수료집계,역무_선후불수익금확정수신,지불반송내역,카드원장,SAM원장,출고등록내역,상태변경내역,출고진행내역,카드일괄판매등록내역,R+_HIPASS카드맵핑명세,RAIL+ 판매(간선),RAIL+ 재고(간선),PL 변경 예외 리스트,고객,고객정보5년간보유,법인회원추가정보,법정대리인,코레일고객,요금과다환불신청내역,카드분실등록내역,고객소유카드등록해제내역,탈퇴고객소유카드등록해제내역,고객안내용원장갱신정보,가입고객,동해선선후불지불거래,동해선선후불지불거래상태,동해선선후불지불거래입력제외,동해선마이비정산일자관리,사업자별 정산결과,동해선선불충전거래,동해선선

In [39]:
input_meta_sepa_tbl

,분리된테이블한글명
0,테이블별시퀀스정보
1,테이블별시퀀스정보
2,서비스설정이력
3,서비스설정이력
4,서비스설정이력
...,...
18736,레일플러스운영현황집계임시
18737,레일플러스운영현황집계임시
18738,레일플러스운영현황집계임시
18739,레일플러스운영현황집계임시


In [52]:
input_meta_sepa_tbl.iloc[:,0]

0            테이블별시퀀스정보
1            테이블별시퀀스정보
2              서비스설정이력
3              서비스설정이력
4              서비스설정이력
             ...      
18736    레일플러스운영현황집계임시
18737    레일플러스운영현황집계임시
18738    레일플러스운영현황집계임시
18739    레일플러스운영현황집계임시
18740    레일플러스운영현황집계임시
Name: 분리된테이블한글명, Length: 18741, dtype: object

In [41]:
pd.DataFrame(columns = input_meta_sepa_tbl.columns)

,분리된테이블한글명


In [81]:
input_meta_sepa_tbl = input_meta_sepa_tbl.fillna('')

In [82]:
emoti_rmv(input_meta_sepa_tbl, '분리된테이블한글명', 'kor')

-----------------------------특수기호제거----------------------------|
index: 144
영업소 관리 -> 영업소관리
-----------------------------특수기호제거----------------------------|
index: 145
영업소 관리 -> 영업소관리
-----------------------------특수기호제거----------------------------|
index: 146
영업소 관리 -> 영업소관리
-----------------------------특수기호제거----------------------------|
index: 147
영업소 관리 -> 영업소관리
-----------------------------특수기호제거----------------------------|
index: 148
영업소 관리 -> 영업소관리
-----------------------------특수기호제거----------------------------|
index: 149
영업소 관리 -> 영업소관리
-----------------------------특수기호제거----------------------------|
index: 150
영업소 관리 -> 영업소관리
-----------------------------특수기호제거----------------------------|
index: 151
영업소 관리 -> 영업소관리
-----------------------------특수기호제거----------------------------|
index: 152
영업소 관리 -> 영업소관리
-----------------------------특수기호제거----------------------------|
index: 153
영업소 관리 -> 영업소관리
-----------------------------특수기호제거----------------------------|
index: 154
영업소 

AttributeError: 'int' object has no attribute 'islower'

In [72]:
input_meta_sepa_tbl['stwd_kor'] = input_meta_sepa_tbl['분리된테이블한글명']
input_meta_sepa_tbl['stwd_kor']

0            테이블별시퀀스정보
1            테이블별시퀀스정보
2              서비스설정이력
3              서비스설정이력
4              서비스설정이력
             ...      
18736    레일플러스운영현황집계임시
18737    레일플러스운영현황집계임시
18738    레일플러스운영현황집계임시
18739    레일플러스운영현황집계임시
18740    레일플러스운영현황집계임시
Name: stwd_kor, Length: 18741, dtype: object

In [77]:
def emoti_rmv(data, col, var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        data['stwd_kor'] = data[col]
        df_lower_kor = pd.DataFrame(columns = data.columns)   
        data['kor_lower'] = ''
        data['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(data)):
            # 영문 소문자 체크 및 기록
            if data.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(data.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(data.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                data.loc[i, 'kor_lower'] = data.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                data.loc[i, 'stwd_kor'] = data.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣_+]', data.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(data.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            data.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', data.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            data.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        data['kor_cor_hist'] = ""
        data['kor_cor_hist'] = data['kor_lower'] + data['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        data_kor_chk_1 = data.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

In [ ]:
globals()["{}".format(nameof(stwd_total))] = rw_data
def emoti_chk(var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(stwd_total)):
        #     특수기호 제거
        #     stwd_total.loc[i, 'stwd_kor'] =  stwd_total.loc[i, 'stwd_kor'].translate(str.maketrans('', '', string.punctuation))

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(stwd_total.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'kor_lower'] = stwd_total.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_kor'] = stwd_total.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', stwd_total.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            stwd_total.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', stwd_total.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            stwd_total.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

    elif var == 'eng':
        # 영문명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_eng = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['eng_lower'] = ''
        stwd_total['eng_ec'] = ''
        eng_chk_list = []
        for i in range(len(stwd_total)):

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_eng'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(stwd_total.loc[i, 'stwd_eng'].upper()))
                eng_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'eng_lower'] = stwd_total.loc[i, 'stwd_eng']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_eng'] = stwd_total.loc[i, 'stwd_eng'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_eng = re.findall('[A-Z0-9+]', stwd_total.loc[i, 'stwd_eng'])
            chk_eng = len(cor_eng) == len(list(stwd_total.loc[i, 'stwd_eng']))

            if bool(chk_eng) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(''.join(cor_eng)))
                eng_chk_list.append(i)

            # 영문대문자, 숫자 외 변경 이력 저장
            stwd_total.loc[i, 'eng_ec'] = re.sub('[A-Z0-9+]', '', stwd_total.loc[i, 'stwd_eng'])        

            # 영문대문자, 숫자 외 자동 삭제
            stwd_total.loc[i, 'stwd_eng'] = ''.join(cor_eng)

        # 변경 이력 저장
        stwd_total['eng_cor_hist'] = ''
        stwd_total['eng_cor_hist'] = stwd_total['eng_lower'] + stwd_total['eng_ec']

        # 적용 후 결과값
        print("====================================영문명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_eng)) + '개')
        stwd_total_eng_chk_1 = stwd_total.iloc[eng_chk_list]
        print("나머지 제거: "+ str(len(eng_chk_list)))
        print('[자동 처리 완료하였습니다]')
        
    else:
        print("잘못된 입력입니다( 'kor' / 'eng' )")

In [55]:
def emoti_rmv(data, col, var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        data['stwd_kor'] = data[col]
        df_lower_kor = pd.DataFrame(columns = data.columns)   
        data['kor_lower'] = ''
        data['kor_ec'] = ''
        kor_chk_list = []
        
        for i in range(len(data)):
            # 영문 소문자 체크 및 기록
            if data.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(data.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(data.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                data.loc[i, 'kor_lower'] = data.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                data.loc[i, 'stwd_kor'] = data.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', data.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(data.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(data.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            data.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', data.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            data.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        data['kor_cor_hist'] = ""
        data['kor_cor_hist'] = data['kor_lower'] + data['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        data_kor_chk_1 = data.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')
        

        print(data)
        print(1)

In [ ]:
globals()["{}".format(nameof(stwd_total))] = rw_data
def emoti_chk(var):
    if var == 'kor':
        # 한글명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_kor = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['kor_lower'] = ''
        stwd_total['kor_ec'] = ''
        kor_chk_list = []
        for i in range(len(stwd_total)):
        #     특수기호 제거
        #     stwd_total.loc[i, 'stwd_kor'] =  stwd_total.loc[i, 'stwd_kor'].translate(str.maketrans('', '', string.punctuation))

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_kor'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_kor = pd.concat([df_lower_kor, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(stwd_total.loc[i, 'stwd_kor'].upper()))
                kor_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'kor_lower'] = stwd_total.loc[i, 'stwd_kor']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_kor'] = stwd_total.loc[i, 'stwd_kor'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_kor = re.findall('[A-Z0-9가-힣+]', stwd_total.loc[i, 'stwd_kor'])
            chk_kor = len(cor_kor) == len(list(stwd_total.loc[i, 'stwd_kor']))

            if bool(chk_kor) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_kor'] + " -> " + str(''.join(cor_kor)))
                kor_chk_list.append(i)

            # 영문대문자, 숫자, 한글 외 변경 이력 저장
            stwd_total.loc[i, 'kor_ec'] = re.sub('[A-Z0-9가-힣+]', '', stwd_total.loc[i, 'stwd_kor'])        

            # 영문대문자, 숫자, 한글 외 자동 삭제
            stwd_total.loc[i, 'stwd_kor'] = ''.join(cor_kor)

        # 변경 이력 저장
        stwd_total['kor_cor_hist'] = ""
        stwd_total['kor_cor_hist'] = stwd_total['kor_lower'] + stwd_total['kor_ec']

        # 적용 후 결과값
        print("====================================한글명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_kor)) + '개')
        stwd_total_kor_chk_1 = stwd_total.iloc[kor_chk_list]
        print("나머지 제거: "+ str(len(kor_chk_list)))
        print('[자동 처리 완료하였습니다]')

    elif var == 'eng':
        # 영문명에 영문 소문자 탐색
        stwd_total = globals()['stwd_total']
        df_lower_eng = pd.DataFrame(columns = stwd_total.columns)        
        stwd_total['eng_lower'] = ''
        stwd_total['eng_ec'] = ''
        eng_chk_list = []
        for i in range(len(stwd_total)):

            # 영문 소문자 체크 및 기록
            if stwd_total.loc[i, 'stwd_eng'].islower():
                tmp = pd.DataFrame(stwd_total.iloc[i,:]).transpose()
                df_lower_eng = pd.concat([df_lower_eng, tmp], axis=0)
                print("----------------------영문 소문자 체크 및 기록--------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(stwd_total.loc[i, 'stwd_eng'].upper()))
                eng_chk_list.append(i)

                # 영문 변경 이력 저장
                stwd_total.loc[i, 'eng_lower'] = stwd_total.loc[i, 'stwd_eng']      

                # 영문 대문자 변경
                stwd_total.loc[i, 'stwd_eng'] = stwd_total.loc[i, 'stwd_eng'].upper()

            # 영문 대문자, 숫자, 한글만 체크
            cor_eng = re.findall('[A-Z0-9+]', stwd_total.loc[i, 'stwd_eng'])
            chk_eng = len(cor_eng) == len(list(stwd_total.loc[i, 'stwd_eng']))

            if bool(chk_eng) == False:
                print("-----------------------------특수기호제거----------------------------|")
                print('index: ' + str(i))
                print(stwd_total.loc[i, 'stwd_eng'] + " -> " + str(''.join(cor_eng)))
                eng_chk_list.append(i)

            # 영문대문자, 숫자 외 변경 이력 저장
            stwd_total.loc[i, 'eng_ec'] = re.sub('[A-Z0-9+]', '', stwd_total.loc[i, 'stwd_eng'])        

            # 영문대문자, 숫자 외 자동 삭제
            stwd_total.loc[i, 'stwd_eng'] = ''.join(cor_eng)

        # 변경 이력 저장
        stwd_total['eng_cor_hist'] = ''
        stwd_total['eng_cor_hist'] = stwd_total['eng_lower'] + stwd_total['eng_ec']

        # 적용 후 결과값
        print("====================================영문명 검토====================================|")
        print('소문자 처리: ' + str(len(df_lower_eng)) + '개')
        stwd_total_eng_chk_1 = stwd_total.iloc[eng_chk_list]
        print("나머지 제거: "+ str(len(eng_chk_list)))
        print('[자동 처리 완료하였습니다]')
        
    else:
        print("잘못된 입력입니다( 'kor' / 'eng' )")

In [8]:
# # 파일 불러오기
# sheet_name = "테이블컬럼정의서"
# input_path = os.path.join(os.getcwd(), "input\\")
# output_path = os.path.join(os.getcwd(), "output\\")
# input_meta = pd.read_excel(input_path+"표준화 INPUT 정의서.xlsx", sheet_name=sheet_name, header=0, na_filter=False)

# # null string ""로 변환
# for i in range(len(input_meta.columns)):
#     input_meta[input_meta.columns[i]] = input_meta[input_meta.columns[i]].fillna("")
# input_meta['기관ID'] = input_meta['물리DB명*'] + "_" + input_meta['관리부서명*']
# print(input_meta.shape)
# input_meta.head(5)

In [9]:
input_meta

,관리부코드*,관리부서명*,시스템명*,물리DB명*,테이블물리명*,테이블한글명*,컬럼물리명*,컬럼물리명FullName,컬럼한글명*,표준도메인그룹명*,도메인분류명*,데이터타입*,데이터길이*
0,,,,,SEQ_TAB,테이블별시퀀스정보,TABLE_NAME,,테이블명,,,VARCHAR2,50
1,,,,,SEQ_TAB,테이블별시퀀스정보,NEXT_ID,,다음시퀀스번호,,,NUMBER,22
2,,,,,SVC_CONF_HIS,서비스설정이력,CONF_ID,,설정ID,,,VARCHAR2,4
3,,,,,SVC_CONF_HIS,서비스설정이력,SVC_ID,,서비스ID,,,VARCHAR2,4
4,,,,,SVC_CONF_HIS,서비스설정이력,CONF_KEY,,설정키,,,VARCHAR2,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18736,,,,,TT_TDA001,레일플러스운영현황집계임시,CUST_ID,,고객ID,,,VARCHAR2,12
18737,,,,,TT_TDA001,레일플러스운영현황집계임시,REG_PS_ID,,등록자ID,,,VARCHAR2,12
18738,,,,,TT_TDA001,레일플러스운영현황집계임시,REG_DTTM,,등록일시,,,CHAR,14
18739,,,,,TT_TDA001,레일플러스운영현황집계임시,CHG_PS_ID,,변경자ID,,,VARCHAR2,12


### 값 검색 로직

In [10]:
# 널 체크
blnk_chk(input_meta)

관리부코드*: 0
관리부서명*: 0
시스템명*: 0
물리DB명*: 0
컬럼물리명FullName: 0
컬럼한글명*: 18479
표준도메인그룹명*: 0
도메인분류명*: 0
blnk(null)이 포함된 컬럼이 탐지되었습니다.
['관리부코드*', '관리부서명*', '시스템명*', '물리DB명*', '컬럼물리명FullName', '컬럼한글명*', '표준도메인그룹명*', '도메인분류명*']
       NULL포함컬럼명  NULL개수
0         관리부코드*  -18741
1         관리부서명*  -18741
2          시스템명*  -18741
3         물리DB명*  -18741
4  컬럼물리명FullName  -18741
5         컬럼한글명*    -262
6      표준도메인그룹명*  -18741
7        도메인분류명*  -18741
null 인덱스를 임시 보관합니다.
널인덱스보관_관리부코드*(변수명) -> null_0
널인덱스보관_관리부서명*(변수명) -> null_1
널인덱스보관_시스템명*(변수명) -> null_2
널인덱스보관_물리DB명*(변수명) -> null_3
널인덱스보관_컬럼물리명FullName(변수명) -> null_4
널인덱스보관_컬럼한글명*(변수명) -> null_5
널인덱스보관_표준도메인그룹명*(변수명) -> null_6
널인덱스보관_도메인분류명*(변수명) -> null_7


In [16]:
input_meta_tbl

,관리부코드*,관리부서명*,시스템명*,물리DB명*,테이블물리명*,테이블한글명*,컬럼물리명*,컬럼물리명FullName,컬럼한글명*,표준도메인그룹명*,도메인분류명*,데이터타입*,데이터길이*
0,,,,,SEQ_TAB,테이블별시퀀스정보,TABLE_NAME,,테이블명,,,VARCHAR2,50
2,,,,,SVC_CONF_HIS,서비스설정이력,CONF_ID,,설정ID,,,VARCHAR2,4
10,,,,,SVC_CONF_INFO,서비스설정정보,SYSTEM_ID,,SYSTEM_ID,,,VARCHAR2,3
23,,,,,SVC_CONF_INFO_HIS,서비스설정정보이력,SYSTEM_ID,,,,,VARCHAR2,3
36,,,,,SVC_CONF_LIST,서비스설정리스트,SYSTEM_ID,,SYSTEM_ID,,,VARCHAR2,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18680,,,,,TT_MTR990,전환 역 정보관리,EXTR_STN_CD,,외부역코드,,,CHAR,4
18685,,,,,TT_MTR991,전환 역 정보관리,EXTR_STN_CD,,외부역코드,,,CHAR,4
18693,,,,,TT_STA999,정류장(임시)테이블,STA_ID,,정류장ID,,,VARCHAR2,10
18696,,,,,TT_TAS311,배분금액검증집계임시,DLVR_DT,,청구일자,,,CHAR,8


In [17]:
len(input_meta_tbl.columns)

13

In [18]:
# 테이블 유니크 선벌(그룹바이)
input_meta_tbl = input_meta.groupby(['테이블물리명*', '테이블한글명*']).head(1)
# 불필요 값 없애기

for i in range(6,14):
    input_meta_tbl.iloc[:,i] = ''
input_meta_tbl

C:\Users\namdd\AppData\Local\Temp\ipykernel_8324\2530784292.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_meta_tbl.iloc[:,i] = ''


,관리부코드*,관리부서명*,시스템명*,물리DB명*,테이블물리명*,테이블한글명*,컬럼물리명*,컬럼물리명FullName,컬럼한글명*,표준도메인그룹명*,도메인분류명*,데이터타입*,데이터길이*
0,,,,,SEQ_TAB,테이블별시퀀스정보,,,,,,,
2,,,,,SVC_CONF_HIS,서비스설정이력,,,,,,,
10,,,,,SVC_CONF_INFO,서비스설정정보,,,,,,,
23,,,,,SVC_CONF_INFO_HIS,서비스설정정보이력,,,,,,,
36,,,,,SVC_CONF_LIST,서비스설정리스트,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18680,,,,,TT_MTR990,전환 역 정보관리,,,,,,,
18685,,,,,TT_MTR991,전환 역 정보관리,,,,,,,
18693,,,,,TT_STA999,정류장(임시)테이블,,,,,,,
18696,,,,,TT_TAS311,배분금액검증집계임시,,,,,,,


In [ ]:
# 불필요 컬럼 빼기
excpt_col = ['컬럼물리명*', '컬럼영문약어명*', '컬럼한글명*', '분리된컬럼한글명*']
excpt_tbl = ['테이블물리명*', '테이블영문약어명*', '테이블한글명*', '분리된테이블한글명*']

# 컬럼명 추림
brief_col(input_meta_tbl, excpt_col, 'pick_tbl')
brief_col(input_meta, excpt_tbl, 'pick_col')

# 용어명 컬럼 추가
pick_tbl['용어명'] = pick_tbl['테이블한글명*']
pick_col['용어명'] = pick_col['컬럼한글명*']
pick_tbl

In [ ]:
# 컬럼명 변경
modify_col = ['물리DB명*', '관리부서명*', '테이블물리명*', '영문약어명', '테이블한글명*', '용어명 분리', 
              '표준도메인 그룹명*', '도메인분류명', '데이터타입*', '데이터길이*', '기관ID', '용어명']
cor_col(pick_tbl, pick_tbl.columns, modify_col, 'pick_tbl')
modify_col = ['물리DB명*', '관리부서명*', '컬럼물리명*', '영문약어명', '컬럼한글명*', '용어명 분리', 
              '표준도메인 그룹명*', '도메인분류명', '데이터타입*', '데이터길이*', '기관ID', '용어명']
cor_col(pick_col, pick_col.columns, modify_col, 'pick_col')

# TBL/COL 기록
pick_col["TBL/COL"] = "COL"
pick_tbl["TBL/COL"] = "TBL"

In [ ]:
# null 제거 및 컬럼(순서) 선택
pick_total = pd.concat([pick_col, pick_tbl]).fillna("")
pick_total_col = ['물리DB명*', '관리부서명*',  '테이블물리명*', '테이블한글명*', '컬럼물리명*', '컬럼한글명*', 
                  '용어명', '용어명 분리', '영문약어명', 
                  '표준도메인 그룹명*', '도메인분류명', '데이터타입*', '데이터길이*', '기관ID', 'TBL/COL']
cor_col(pick_total, pick_total_col, pick_total_col, 'pick_total')
# 인덱스 초기화
pick_total = pick_total.reset_index(drop=True)
print(pick_total.shape)
pick_total.head()

#### . 파일 불러오기

In [ ]:
# # 파일 불러오기
# # test_emoti_dup_korDup
# sheet_name = "input_test"
# input_path = os.path.join(os.getcwd(), "input\\")
# output_path = os.path.join(os.getcwd(), "output\\")
# # input_test = pd.read_excel(input_path+"test_emoti_dup_korDup.xlsx", sheet_name=sheet_name, header=0)
# input_test = pd.read_excel(input_path+"test_final.xlsx", sheet_name=sheet_name, header=0)
# # test_final.xlsx
# # null 제거
# for i in range(len(input_test.columns)):
#     input_test[input_test.columns[i]] = input_test[input_test.columns[i]].fillna("")
# print(input_test.shape)
# input_test.head(5)

### 추가 1) 메타데이터 띄움처리 전 중복 제거..
### 추가 2) 한글명 notnull vs null 분리하여 notnull만 관리(null: 파일 생성 및 기입 요청)
### 추가 3.1) 한글 용어 점검 기능: 한글명+영문명+숫자조합 가능), 그외 모두 제거
### 추가 3.2) 한글 단어 점검 기능: 한글명(숫자조합 가능), 영문명만(숫자조합 가능) 그외 모두 제거
### 추가 4) 테이블기준 상세항목 정의하기.. 

# 2. 용어 분리

### 용어명 변경